# Assignment 1 - part 2
### Build our first neural network

*first step: read the data and split it to train and validation*

In [7]:
import pandas as pd
import numpy as np
from scipy import misc
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
nrow = 224
ncol = 224
channels = 3

##### read the data:
by using get_dummies we invert the df to be categorial, each index is an image

In [8]:
train_df = pd.read_csv('data/train_relevant.csv', index_col=0) # index_col replace the index with the image_name
train_df_categorial = pd.get_dummies(data=train_df,columns=['whaleID'])

##### split the data to data_X and data_Y:  
data_X - an numpy array of images.  
data_Y - matrix with 0's and 1's , each row only one column filled with 1 for the matching whale.

In [9]:
data_Y = train_df_categorial.as_matrix().astype(np.float32) # for each image there will be an array with binary result about each kind of whale
num_of_images = train_df.shape[0]
data_X = np.zeros(shape=(num_of_images,nrow,ncol,channels), dtype=np.float32)
for index,img_name in enumerate(train_df.index):
    image = misc.imread('trainCrops/' + img_name)
    image = misc.imresize(image,size=(nrow,ncol,channels))
    data_X[index] = image.astype(np.float32)

c:\users\ariel\anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
c:\users\ariel\anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
c:\users\ariel\anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


as we saw at part 1, there are whales with only 1 image. we will seperate them from the data, and decide later how to treat this group of whales.  
rows_belong_to_classify_with_one_img are indeces of images of whales that has only 1 image.

In [10]:
rows_belong_to_classify_with_one_img = []
for index, row in enumerate(data_Y):
    x = np.nonzero(row) # get indexes in the row that are not zero in each row only 1 index that differ from 0 exist.
    if data_Y[:,x].sum() == 1:
        rows_belong_to_classify_with_one_img.append(index)
rows_belong_to_classify_with_one_img

[41,
 319,
 352,
 863,
 884,
 1010,
 1060,
 1118,
 1184,
 1227,
 1374,
 1590,
 1606,
 1730,
 2058,
 2091,
 2365,
 2916,
 3033,
 3097,
 3613,
 3671,
 3835,
 3881]

use 3 fold cross validation with stratified in order to keep the proportion of samples in the train and validation the same

In [11]:
y = train_df.reset_index()[['whaleID']].as_matrix()
num_of_splits = 3
skf = StratifiedKFold(n_splits=num_of_splits, shuffle=False)

c:\users\ariel\anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


define cnn:

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D

Using TensorFlow backend.


In [ ]:
num_classes = 447
def create_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(nrow,ncol,channels)))
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(MaxPool2D())
    model.add(Conv2D(16,(3,3),activation='relu'))
    model.add(Conv2D(16,(3,3),activation='relu'))
    model.add(MaxPool2D())
    model.add(MaxPool2D())

    model.add(Flatten())
    model.add(Dense(447,activation='softmax'))
    return model

In [ ]:
def train_model(model, xtrain, ytrain, xval, yval):
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,validation_data=[xval,yval],epochs=10)

In [ ]:
for index, (train_indices, val_indices) in enumerate(skf.split(data_X, y)):
    print('Training on fold ' + str(index+1) + "/3...")
    xtrain, xval = data_X[train_indices]/255, data_X[val_indices]/255
    ytrain, yval = data_Y[train_indices], data_Y[val_indices]
    
    model = None
    model = create_model()
    print(model.summary())
    history = train_model(model, xtrain, ytrain, xval, yval)
    #accuracy_history = history.history['acc']
    #val_accuracy_history = history.history['val_acc']
    #print('Last training accuracy: ' + str(accuracy_history[-1]) + ', last validation accuracy: ' + str(val_accuracy_history[-1]))

c:\users\ariel\anaconda3\envs\workshop\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Training on fold 1/3...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 16)      4624      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 16)     